In [15]:
from pyspark.sql.types import *
from pyspark.sql import Row

In [2]:
# Read in raw ratings data (fromUserId, toUserId, rating)
ratingsCsvRDD = sc.textFile("s3n://insight-spark-after-dark/ratings.csv.gz").repartition(16)

In [18]:
# Convert raw ratings RDD to Json RDD
def rec_tup(rating):
    tokens = rating.split(",")
    return Row(fromUserId=int(tokens[0]), toUserId=int(tokens[1]), rating=int(tokens[2]))

ratingsJson_DF = ratingsCsvRDD.map(rec_tup).toDF()
ratingsJson_DF.take(5)

[Row(fromUserId=1, rating=8, toUserId=2145),
 Row(fromUserId=1, rating=7, toUserId=11747),
 Row(fromUserId=1, rating=4, toUserId=18878),
 Row(fromUserId=1, rating=4, toUserId=23499),
 Row(fromUserId=1, rating=10, toUserId=35647)]

In [19]:
# Cache the SchemaRDD as we'll be using this heavily moving forward
ratingsJson_DF.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[fromUserId: bigint, rating: bigint, toUserId: bigint]

In [20]:
# Describe the SchemaRDD inferred from the JSON
ratingsJson_DF.printSchema()

ratingsJson_DF.registerTempTable("ratingsJsonTable")

root
 |-- fromUserId: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- toUserId: long (nullable = true)



In [21]:
sqlContext.sql("DESCRIBE ratingsJsonTable")

DataFrame[col_name: string, data_type: string, comment: string]

In [22]:
# Show the top 10 most-active users
mostActiveUsersSchemaRDD = sqlContext.sql("""
    SELECT * 
    FROM ratingsJsonTable 
    """)

In [ ]:
mostActiveUsersSchemaRDD.collect()

In [ ]:
ratingsJsonSchema_df.sample(False, 0.1, 20).collect()